In [1]:
!pip install -q transformers datasets peft adapters rouge-score

In [2]:
!pip install torch

In [3]:
!pip install --upgrade wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 59.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.16.6
    Uninstalling wandb-0.16.6:
      Successfully uninstalled wandb-0.16.6


In [4]:
!pip install -U -q git+https://github.com/huggingface/trl

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import torch
import pandas as pd
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM, AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer, AdamW
from trl import DPOTrainer, DPOConfig
from transformers import BitsAndBytesConfig
from peft import PeftModel
from datasets import Dataset, load_dataset
from peft import get_peft_config
from sklearn.model_selection import train_test_split
from trl import DPOTrainer  # Импортируем DPOTrainer из trl
from peft import PeftModel  # Импортируем PeftModel
from safetensors.torch import load_model, save_model

2024-05-23 21:22:06.691456: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 21:22:06.691551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 21:22:06.835611: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

set_random_seed(69)

In [7]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [8]:
torch.cuda.empty_cache()

In [9]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:256'

In [26]:
# Загрузка основной модели и токенизатора

model = AutoModelForCausalLM.from_pretrained('ai-forever/rugpt3medium_based_on_gpt2', device_map="auto")
tokenizer = AutoTokenizer.from_pretrained('ai-forever/rugpt3medium_based_on_gpt2')

#model_name = "cointegrated/rut5-base-multitask"
#tokenizer = T5Tokenizer.from_pretrained(model_name)
#model = T5ForConditionalGeneration.from_pretrained(model_name)
model.config.use_cache = False

# Загрузка эталонной модели и замораживание её параметров
ref_model = AutoModelForCausalLM.from_pretrained('ai-forever/rugpt3medium_based_on_gpt2')
#ref_model = T5ForConditionalGeneration.from_pretrained(model_name)
for param in ref_model.parameters():
    param.requires_grad = False

In [27]:
# Настройка конфигурации PEFT для задачи автоматического исправления текста
config = {
    "peft_type": "PREFIX_TUNING",  # Тип PEFT (Prefix Tuning)
    "task_type": "SEQ2SEQ_LM",  # Тип задачи (Sequence to Sequence Language Model)
    "inference_mode": False,  # Режим инференса (False для обучения)
    "num_virtual_tokens": 20,  # Количество виртуальных токенов
    "token_dim": 128,  # Размерность токена
    "num_transformer_submodules": 1,  # Количество подмодулей трансформера
    "num_attention_heads": 12,  # Количество голов внимания
    "num_layers": 12,  # Количество слоев
    "encoder_hidden_size": 128,  # Размер скрытого слоя энкодера
    "prefix_projection": False,  # Проекция префикса (False для базовой настройки)
}


peft_config = get_peft_config(config)
# Создание PeftModel на основе основной модели
model = PeftModel(model, peft_config)
for param in model.parameters():
    param.requires_grad = True

In [28]:
# Загрузка и обработка данных
data = pd.read_csv('/kaggle/input/sentences-preprocessed/sentences_preprocessed.csv')
data = data[['text', 'corrected']].dropna()
data['input_token_len'] = data['text'].apply(calc_token_len)
data['input_token_len'].quantile(0.95), data['input_token_len'].quantile(0.05)

(37.0, 4.0)

In [29]:
data = data[(data['input_token_len'] <= 37) & (data['input_token_len'] > 4)]
data['chosen'] = 'Исправьте все грамматические и орфографические ошибки в этом предложении: "' + data['text'] + '" Исправленное предложение: ' + data['corrected']
data['rejected'] = 'Исправьте все грамматические и орфографические ошибки в этом предложении: "' + data['text'] + '" Исправленное предложение: ' + "ничего"
data['text'] = 'Исправьте все грамматические и орфографические ошибки в этом предложении: "' + data['text'] + '" Исправленное предложение: '
train_df, test_df = train_test_split(data, test_size=0.3, shuffle=False)
val_df, test_df = train_test_split(test_df, test_size=0.5, shuffle=False)

In [32]:
data['chosen'][0]

'Исправьте все грамматические и орфографические ошибки в этом предложении: "Загрязнение тяжелыми металлами Дальнозоркого района ." Исправленное предложение: Загрязнение тяжелыми металлами Дальнегорского района .'

In [33]:
train_df.shape, val_df.shape, test_df.shape

((18907, 5), (4051, 5), (4052, 5))

In [34]:
# Определение конфигураций обучения
training_args = DPOConfig(num_train_epochs=1, output_dir="./output-dir", max_length=84)

# Подготовка DPO наборов данных
train_dpo_dataset = {
    "prompt": train_df['text'].tolist(),
    "chosen": train_df['chosen'].tolist(),
    "rejected": train_df['rejected'].tolist()
}

eval_dpo_dataset = {
    "prompt": val_df['text'].tolist(),
    "chosen": val_df['chosen'].tolist(),
    "rejected": val_df['rejected'].tolist()
}

train_dataset = Dataset.from_dict(train_dpo_dataset)
eval_dataset = Dataset.from_dict(eval_dpo_dataset)

save_model(model, "model.safetensors")


dpo_trainer = DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)


dpo_trainer.train()

Map:   0%|          | 0/18907 [00:00<?, ? examples/s]

Map:   0%|          | 0/4051 [00:00<?, ? examples/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
500,0.285200
1000,0.274100
1500,0.263600
2000,0.274800


TrainOutput(global_step=2364, training_loss=0.2735280877644237, metrics={'train_runtime': 1992.2864, 'train_samples_per_second': 9.49, 'train_steps_per_second': 1.187, 'total_flos': 0.0, 'train_loss': 0.2735280877644237, 'epoch': 1.0})

In [35]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [72]:
model.eval()

def generate_correction(sentence, max_length=85, temperature=1.5):
    inputs = tokenizer.encode(sentence, truncation=True, max_length=max_length, return_tensors='pt')
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    outputs = model.generate(inputs, num_beams=4, max_length=max_length, num_return_sequences=1, temperature=temperature)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

In [73]:
test_df['corrected_by_dpo'] = test_df['text'].progress_apply(generate_correction)

  0%|          | 0/4052 [00:00<?, ?it/s]

In [74]:
test_df.to_csv('/kaggle/working/dpo_rugpt3_results.csv')